In [5]:
# basic setup for error correcting code
# modeled with a markov chain

# start simlpe, we will add complexity later

# define a codespace
# "good" states have probability of faults

# different type of faults
# bit flip, phase flip

# different states for number of faults
# if too many, is a terminal state (uncorrectable)

# fault states have a probability of recovery
# unclear to me if the recovery should be a single probability,
# or since requires faulty gates, should be more states
# i.e. can break it into syndrome correctly detected, and recovery correctly performed

# have probability of going from fault state to another fault state

# what is expected number of operations we can do before reaching a terminal state?

# want to use rustworkx.PyDiGraph to model the graph

In [6]:
# basic example
# can handle 3 faults before uncorrectable
# every operation as a 1-p rate of a fault
# p^k for k faults on a single operation
# correcting a fault takes 2*k operations

# what is the expected number of operations can do before reach uncorrectable state

# here p is probability of success
# (out, in, weight)
# [correct, f1, p]
# [correct, f2, p^2]
# [correct, f3, p^3]
# [correct, f4, p^4]
# [f1, correct, p^(2*k)]
# [f1, f1, p^?]
# ....
# [f4, f4, 1]

In [7]:
from rustworkx import PyDiGraph
import numpy as np


class MarkovModel(PyDiGraph):
    @classmethod
    def from_edge_list(cls, edge_list):
        """Construct MarkovModel from edge list."""
        model = cls()
        node_mapping = {}

        for src, dest, prob in edge_list:
            if src not in node_mapping:
                node_mapping[src] = model.add_node(src)
            if dest not in node_mapping:
                node_mapping[dest] = model.add_node(dest)

            model.add_edge(node_mapping[src], node_mapping[dest], prob)

        return model

    @classmethod
    def from_transition_matrix(cls, transition_matrix):
        """Construct MarkovModel from transition matrix."""
        model = cls()
        states = range(len(transition_matrix))

        for src in states:
            for dest in states:
                prob = transition_matrix[src][dest]
                if prob > 0:  # Only add non-zero probability transitions
                    model.add_edge(src, dest, prob)

        return model

    @property
    def transition_matrix(self):
        """Get transition matrix from graph representation."""
        num_nodes = self.number_of_nodes()
        matrix = np.zeros((num_nodes, num_nodes))

        for edge in self.edge_data():
            src, dest, prob = edge
            matrix[src][dest] = prob

        return matrix

    @property
    def eigenstates(self):
        """Compute the eigenstates of the transition matrix."""
        matrix = self.transition_matrix
        _, eigenvectors = np.linalg.eig(matrix)
        return eigenvectors


class ModelCircuit(MarkovModel):
    def __init__(self, fault_probabilities, max_faults):
        super().__init__()
        self.fault_probabilities = fault_probabilities
        self.max_faults = max_faults

In [11]:
class ModelCircuit(MarkovChainGraphUndirectedUnweighted):
    def __init__(self, fault_probabilities, max_faults):
        super().__init__()
        self.fault_probabilities = fault_probabilities
        self.max_faults = max_faults
        self._initialize_graph()

    def _initialize_graph(self):
        # Add nodes for each fault level, including "correct" and terminal state "f4"
        for i in range(self.max_faults + 1):
            self.add_node(f"f{i}")

        # Define transitions based on fault probabilities
        for i in range(self.max_faults):
            current_state = f"f{i}"
            next_state = f"f{i+1}"

            # Transitions for new faults
            self.add_transition_probability(
                current_state, next_state, self.fault_probabilities[i]
            )

            # Transitions for recovery circuit
            for j in range(1, i + 2):  # considering all possible faults from 1 to i+1
                recovery_success_prob = (1 - self.fault_probabilities[0]) ** (
                    2 * j
                )  # 2*j operations for j faults
                if j <= i:
                    self.add_transition_probability(
                        current_state, f"f{j}", recovery_success_prob
                    )
                else:
                    self.add_transition_probability(
                        current_state, next_state, recovery_success_prob
                    )

        # Terminal state always remains in terminal state
        self.add_transition_probability(f"f{self.max_faults}", f"f{self.max_faults}", 1)


# Using the given example probabilities to test the ModelCircuit class
fault_probabilities = [0.1, 0.01, 0.001]
max_faults = 3
circuit = ModelCircuit(fault_probabilities, max_faults)

# Simulate the transitions from a corrected state for 5000 steps
result = get_transitions(
    f"f0",
    [
        (src, dest, circuit.connections[src][dest])
        for src in circuit.get_nodes()
        for dest in circuit.connections[src]
    ],
    9000,
)
result

Counter({'f3': 8999, 'f1': 2, 'f2': 2, 'f0': 1})